# 快速入门
本部分介绍了机器学习中常见任务的API


## 处理数据
PyTorch处理数据有两个基本组件
* `torch.utils.data.DataLoader`
* `torch.utils.data.Dataset`

`Dataset`存储样本及其对应标签，而`DataLoader`将可迭代对象包装在`Dataset`周围

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch 提供了特定领域的库，如 `TorchText`、`TorchVision` 和 `TorchAudio`，所有这些库都包含数据集。在本教程中，使用一个 `TorchVision` 数据集。

`torchvision.datasets`模块包含许多现实世界视觉数据的`Dataset`对象，如 CIFAR、COCO([完整列表在这里](https://pytorch.org/vision/stable/datasets.html))。在这里使用FashionMNIST数据集。

每个TorchVision`Dataset`包含两个参数：`transform`和`target_transform`，分别用于修改样本和标签。

In [ ]:
# 从开放的数据集中下载训练数据集
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor() # 将图像转换成张量的形式
)

# 从开放的数据集下载测试数据集
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [02:15<00:00, 195118.65it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 115187.18it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1909830.73it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 34273455.54it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



下面将`Dataset`作为参数传递给`DataLoader`。这会对我们的数据集进行可迭代包装，并支持自动批处理、采样、洗牌和多进程数据加载。在这里，定义了一个批处理大小为64，级数据加载器可迭代中的每个元素将返回64个特征和标签的批次

In [4]:
batch_size = 64

# 创建数据加载器(data loaders)
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# 看一下test_dataloader中的样本和标签的形状
for X, y in test_dataloader:
    print(f"样本X的形状[N, C, H, W]: {X.shape}")
    print(f"标签y的形状和数据类型：{y.shape} {y.dtype}")
    break

样本X的形状[N, C, H, W]: torch.Size([64, 1, 28, 28])
标签y的形状和数据类型：torch.Size([64]) torch.int64


## 创建模型
要在PyTorch中定义神经网络，需要创建一个继承`nn.Module`的类。在`__init__`函数中定义网络的层，并在`forward`函数中指定数据如何通过网络传递。为了加速神经网络中的操作，将网络移动到GPU上。

In [9]:
# 为训练分配GPU
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"可用的设备：{device}")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # 调用父类的初始化方法
        self.flatten = nn.Flatten() # 定义一个Flatten层，用于将输入展平
        self.linear_relu_stack = nn.Sequential( # 定义一个顺序容器。包含线性层和ReLU激活函数
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x) # 将x进行展平
        logits = self.linear_relu_stack(x) # 将展平后的输入通过顺序容器处理
        return logits # 返回最终的处理结果logits
    

model = NeuralNetwork().to(device) # 创建NeuralNetwork的实例，并将其移动到指定的GPU
print(model) # 这个是打印模型的结构


可用的设备：cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 优化模型参数
要训练一个模型，需要一个损失函数和一个优化器

In [11]:
loss_fn = nn.CrossEntropyLoss() # 损失函数使用交叉熵损失函数，也要搞清楚内部的变量
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # 优化函数使用Adam函数，请了解Adam函数内部的变量

在单个的训练循环中，模型对训练数据集（以批次形式输入）进行预测，并反向传播预测误差以调整模型的参数

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    # 定义一个名为 train 的函数，该函数接受四个参数：
    # dataloader: 包含训练数据的加载器，每次迭代提供一个小批量的数据。
    # model: 需要训练的神经网络模型。
    # loss_fn: 计算模型预测值与真实标签之间误差的损失函数。
    # optimizer: 用于更新模型参数的优化器（例如SGD或Adam）。
    size = len(dataloader.dataset) # 获取整个训练数据集的大小，存储在变量 size 中。
    model.train() # 将模型设置为训练模式。对于某些层（如Dropout和BatchNorm），训练模式和评估模式会有不同的行为。
    for batch, (X, y) in enumerate(dataloader):
        # 使用 for 循环遍历 dataloader，其中 batch 是当前迭代的索引，(X, y) 是当前小批量的输入数据和对应的标签。
        # X: 当前小批量的输入数据。
        # y: 当前小批量的真实标签。
        X, y = X.to(device), y.to(device) # 将输入数据 X 和标签 y 迁移到指定的计算设备（如GPU或CPU），以加速训练过程。

        # 计算预测误差
        pred = model(X) # 通过模型进行前向传播计算，得到预测结果 pred。
        loss = loss_fn(pred, y) # 使用损失函数计算预测结果 pred 与真实标签 y 之间的误差，并将误差值存储在 loss 中。

        # 反向传播更新参数
        loss.backward() # 计算损失相对于模型参数的梯度，为反向传播。
        optimizer.step() # 使用优化器更新模型参数。
        optimizer.zero_grad() # 清空优化器中存储的梯度，防止梯度累积，为下一次迭代做准备。

        if batch % 100 == 0: 
            # 每隔 100 个小批量时，执行打印操作。
            # 这里使用取模运算符 % 来判断当前 batch 是否是 100 的倍数。
            loss, current = loss.item(), (batch + 1) * len(X) 
            # 将 loss 转换为 Python 浮点数，并存储在 loss 中。
            # 计算当前已经处理的数据量，存储在 current 中。
            # (batch + 1) * len(X) 表示已经处理了 (batch + 1) 个小批量，
            # 每个小批量包含 len(X) 条数据。
            print(f"损失：{loss:>7f} [{current:>5d} / {size:>5d}]")
            # 打印当前损失值以及训练进度。
            # :>7f 表示浮点数右对齐且保留七位小数。
            # :>5d 表示整数右对齐，最小宽度为五位。
            # 这行代码的输出格式为：损失：XXXX.XXXXXX [YYYYY / ZZZZZ]
            # 其中 XXXX.XXXXXX 是当前损失值，YYYYY 是当前已经处理的数据量，
            # ZZZZZ 是整个训练数据集的大小。

下面检查模型在测试数据集上的表现

In [16]:
def test(dataloader, model, loss_fn):
    # 计算数据集的大小
    size = len(dataloader.dataset)
    # 计算数据加载器的批次数量
    num_batches = len(dataloader)
    # 将模型设置为评估模式
    model.eval()
    # 初始化测试损失和正确预测的数量
    test_loss, correct = 0, 0
    with torch.no_grad():
        # 遍历数据加载器中的每个批次
        for X, y in dataloader:
            # 将输入数据和标签移动到指定设备
            X, y = X.to(device), y.to(device)
            # 使用模型进行预测
            pred = model(X)
            # 累加损失值
            test_loss += loss_fn(pred, y).item()
            # 计算正确预测的数量
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # 计算平均测试损失
    test_loss /= num_batches
    # 计算平均准确率
    correct /= size
    # 打印测试误差、准确率和平均误差
    print(f"测试误差：\n 准确率： {(100 * correct):>0.1f}%, 平均误差: {test_loss:>8f} \n")

训练过程分为多个迭代（轮次）。在每个轮次中，模型学习参数以做出更好的预测。我们在每个轮次打印模型的准确率和损失；我们希望看到准确率随着每个轮次的增加而提高，损失随着每个轮次的增加而降低。

In [24]:
epochs = 20
for t in range(epochs):
    print(f"轮次 {t + 1} \n ----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn, )

print("训练完成！")

轮次 1 
 ----------------------------
损失：0.083336 [   64 / 60000]
损失：0.194866 [ 6464 / 60000]
损失：0.179838 [12864 / 60000]
损失：0.339022 [19264 / 60000]
损失：0.256714 [25664 / 60000]
损失：0.330778 [32064 / 60000]
损失：0.138128 [38464 / 60000]
损失：0.259751 [44864 / 60000]
损失：0.149342 [51264 / 60000]
损失：0.119077 [57664 / 60000]
测试误差：
 准确率： 89.1%, 平均误差: 0.479899 

轮次 2 
 ----------------------------
损失：0.083336 [   64 / 60000]
损失：0.194866 [ 6464 / 60000]
损失：0.179838 [12864 / 60000]
损失：0.339022 [19264 / 60000]
损失：0.256714 [25664 / 60000]
损失：0.330778 [32064 / 60000]
损失：0.138128 [38464 / 60000]
损失：0.259751 [44864 / 60000]
损失：0.149342 [51264 / 60000]
损失：0.119077 [57664 / 60000]
测试误差：
 准确率： 89.1%, 平均误差: 0.479899 

轮次 3 
 ----------------------------
损失：0.083336 [   64 / 60000]
损失：0.194866 [ 6464 / 60000]
损失：0.179838 [12864 / 60000]
损失：0.339022 [19264 / 60000]
损失：0.256714 [25664 / 60000]
损失：0.330778 [32064 / 60000]
损失：0.138128 [38464 / 60000]
损失：0.259751 [44864 / 60000]
损失：0.149342 [51264 / 60000]
损失：0.119

## 保存模型
常见方法是序列化内部状态字典（包含模型参数）。

In [25]:
torch.save(model.state_dict(), "model.pth")
print("已将 PyTorch 模型状态保存至 model.pth")

已将 PyTorch 模型状态保存至 model.pth


## 加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载其中。

In [26]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

现在该模型可以用于进行预测

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'预测: "{predicted}", 真实值: "{actual}"')

预测: "Ankle boot", 真实值: "Ankle boot"


在这里了解更多有关保存和加载模型的信息[Saving&Loading_model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)